In [13]:
import pandas as pd

data = []

with open("data/app.log", "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        timestamp = line[:19]          # '2024-11-08 10:00:00'
        rest = line[20:]               # resto da linha

        level, message = rest.split(" ", 1)

        data.append({
            "timestamp": pd.to_datetime(timestamp),
            "level": level
        })

df = pd.DataFrame(data)

In [14]:
df["minute"] = df["timestamp"].dt.floor("min")

errors_per_minute = (
    df[df["level"] == "ERROR"]
    .groupby("minute")
    .size()
    .reset_index(name="error_count")
)

errors_per_minute.head()

,minute,error_count
0,2024-11-08 10:00:00,5
1,2024-11-08 10:01:00,2
2,2024-11-08 10:02:00,2
3,2024-11-08 10:03:00,5
4,2024-11-08 10:04:00,2


In [15]:
from sklearn.linear_model import LinearRegression
import numpy as np

errors_per_minute["t"] = np.arange(len(errors_per_minute))

X = errors_per_minute[["t"]]
y = errors_per_minute["error_count"]


In [19]:
future_t = pd.DataFrame(
    np.arange(len(errors_per_minute), len(errors_per_minute) + 10),
    columns=["t"]
)

predictions = model.predict(future_t)
print(predictions)

[5.00189749 5.00189704 5.00189659 5.00189614 5.0018957  5.00189525
 5.0018948  5.00189436 5.00189391 5.00189346]


In [22]:
THRESHOLD = 5  # erros por minuto

if predictions.max() > THRESHOLD:
    print("ALERTA: Possível pico de erros iminente")

ALERTA: Possível pico de erros iminente
